# Read-only Postgres user

Creating a read-only Postgres user to be able to query the database backend


In [2]:
import psycopg2 as pg
from psycopg2 import sql, errors
import pandas as pd

Pg connection parameters

In [3]:
conn_params = {
    'dbname': "eamena",
    'user': "eamenar_temp",
    'password': "",
    'host': "52.50.27.140",
    'port': "5432"
}

Connect the DB

In [4]:
try:
    connection = pg.connect(**conn_params)
    cursor = connection.cursor()
    print("Connection established successfully!")
except pg.Error as e:
    print(f"Error: {e}")

Connection established successfully!


## Tests

After granting `SELECT` permission (i.e. read-only) on the single view `nb_hp_by_gs` to `eamenar_temp` user (see: [User permissions](https://github.com/eamena-project/eamena-arches-dev/tree/main/dbs/database.eamena/postgres/queries#user-permissions))

### ✅ Read the authorised View

Try to remove the authorised view (possible)

In [5]:
pd.read_sql("SELECT * FROM nb_hp_by_gs LIMIT 10;", connection)

<ipython-input-5-0e5b0a5ecf5b>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT * FROM nb_hp_by_gs LIMIT 10;", connection)


,grid_id,nb_hp,grid_num
0,36997a90-8e1c-4693-9738-1af1b428a07c,5,E34N31-13
1,9786a8b2-4f69-46c9-a0b0-656c26e5f908,78,E34N31-14
2,8b54b4d1-2077-4594-b3e6-59b5c9bbf3ae,21,E34N31-42
3,be51ee3c-d2aa-4244-ae69-f93cca04a690,118,E35N31-31
4,09107442-c267-44cb-806f-d66ea5d2d70b,1,E34N31-12
5,c4e8738c-c3a5-4aba-a3f5-31f3890150e0,20,E34N31-32
6,c077f029-3185-4177-bac5-81f14846595f,7,E34N32-22
7,ab1ffdfe-2e6e-4502-8080-ba2d705af546,231,E35N31-34
8,5274cb77-b93b-465c-9323-7e5deda2d01c,86,E35N32-12
9,db704a72-ddf0-4139-963c-2eb5cccb3b0e,20,E34N31-24


### ❌ Read another View

Try to read another View (impossible)

In [6]:
try:
  pd.read_sql("SELECT * FROM nb_hp_by_gs_temp LIMIT 10;", connection)
  print("Table can be read :(")
except pd.io.sql.DatabaseError as e:
    print("Insufficient privileges to perform this operation :)")
    print(f"Error message: {e}")

Insufficient privileges to perform this operation :)
Error message: Execution failed on sql 'SELECT * FROM nb_hp_by_gs_temp LIMIT 10;': permission denied for view nb_hp_by_gs_temp



<ipython-input-6-0007a05c2a23>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT * FROM nb_hp_by_gs_temp LIMIT 10;", connection)


### ❌ Read a Table

Try to read a table (impossible)

In [7]:
connection = pg.connect(**conn_params)
cursor = connection.cursor()
sqll = sql.SQL("SELECT * FROM auth_group_temp LIMIT 10")
try:
  cursor.execute(sqll)
  connection.commit()
  print("Table can be read :(")
except errors.InsufficientPrivilege as e:
    print("Insufficient privileges to perform this operation :)")
    print(f"Error message: {e}")

Insufficient privileges to perform this operation :)
Error message: permission denied for table auth_group_temp



### ❌ Delete a Table

Try to remove a table (impossible)

In [8]:
connection = pg.connect(**conn_params)
cursor = connection.cursor()
sqll = sql.SQL("DROP TABLE IF EXISTS auth_group_temp")
try:
  cursor.execute(sqll)
  connection.commit()
  print("Table has been dropped :(")
except errors.InsufficientPrivilege as e:
    print("Insufficient privileges to perform this operation :)")
    print(f"Error message: {e}")

Insufficient privileges to perform this operation :)
Error message: must be owner of table auth_group_temp



### Close connection

In [9]:
cursor.close()
connection.close()

## Documentation

- [User permissions](https://github.com/eamena-project/eamena-arches-dev/tree/main/dbs/database.eamena/postgres/queries#user-permissions)